In [1]:
import re
import numpy as np
import pandas as pd
import re
import numpy as np
import re
import pandas as pd
from textblob import TextBlob
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import numpy as np
import re
import pandas as pd
from textblob import TextBlob
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

df = pd.read_csv('Iphone14Reviews.csv')

In [2]:
df.columns

Index(['Unnamed: 0', 'Product', 'Review', '5 Stars', '4 Stars', '3 Stars',
       '2 Stars', '1 Stars', 'Avg Rating', 'Price', 'Link'],
      dtype='object')

In [3]:
# df.drop('Unnamed: 0.2', axis = 1, inplace = True)
# df.drop('Unnamed: 0.1', axis = 1, inplace = True)
df.drop('Unnamed: 0', axis = 1, inplace = True)
# df.drop('Unnamed: 0.4', axis = 1, inplace = True)
# df.drop('Unnamed: 0.4', axis = 1, inplace = True)
#df.drop('Unnamed: 0.3', axis = 1, inplace = True)
#df.drop('Cleaned', axis = 1, inplace = True)

In [4]:
df

,Product,Review,5 Stars,4 Stars,3 Stars,2 Stars,1 Stars,Avg Rating,Price,Link
0,"Apple iPhone 14 (128GB, Blue)",Its a very good product.,172,20,2,4,6,['4.7 '],"[' ₹70,990.00 ']",https://www.croma.com/apple-iphone-14-128gb-bl...
1,"Apple iPhone 14 (128GB, Blue)",So far good,172,20,2,4,6,['4.7 '],"[' ₹70,990.00 ']",https://www.croma.com/apple-iphone-14-128gb-bl...
2,"Apple iPhone 14 (128GB, Blue)",Wonderful,172,20,2,4,6,['4.7 '],"[' ₹70,990.00 ']",https://www.croma.com/apple-iphone-14-128gb-bl...
3,"Apple iPhone 14 (128GB, Blue)",Good,172,20,2,4,6,['4.7 '],"[' ₹70,990.00 ']",https://www.croma.com/apple-iphone-14-128gb-bl...
4,"Apple iPhone 14 (128GB, Blue)",Very good performance,172,20,2,4,6,['4.7 '],"[' ₹70,990.00 ']",https://www.croma.com/apple-iphone-14-128gb-bl...
...,...,...,...,...,...,...,...,...,...,...
2992,Apple iPhone 14 Pro Max (1 TB) - Space Black,NaN,49%,8%,21%,0%,22%,3.6 out of 5,"₹1,89,900",https://www.amazon.in/Apple-iPhone-Pro-Space-B...
2993,Apple iPhone 14 Pro Max (1 TB) - Space Black,This is my Personal & primary (main) MobileNew...,49%,8%,21%,0%,22%,3.6 out of 5,"₹1,89,900",https://www.amazon.in/Apple-iPhone-Pro-Space-B...
2994,Apple iPhone 14 Pro Max (1 TB) - Space Black,Insane battery life . I'm happy with purchase....,49%,8%,21%,0%,22%,3.6 out of 5,"₹1,89,900",https://www.amazon.in/Apple-iPhone-Pro-Space-B...
2995,Apple iPhone 14 Pro Max (1 TB) - Space Black,"Specifications good,but seems repeated you may...",49%,8%,21%,0%,22%,3.6 out of 5,"₹1,89,900",https://www.amazon.in/Apple-iPhone-Pro-Space-B...


In [5]:
class Cleancsv:
    def __init__(self, df):
        self.df = df
    
    def clean(self, df):
        x = self.df['Avg Rating']
        for i, j in enumerate(x):
            if type(j) == list():
                j = j[0]
                y = re.sub(r'[^0-9.]+','', j)
                y = y+ '   '
                y = y[:3].strip()
                self.df.loc[i, 'Avg Rating'] = y
            else:
                if type(j) == str:
                    y = re.sub(r'[^0-9.]+','', j)
                    y = y+ '   '
                    y = y[:3].strip()
                    self.df.loc[i, 'Avg Rating'] = y


        # cleeaning price
        x = self.df['Price']
        index = 0
        for i in x:
            if type(i) == list:
                y = i[0]
                y = y.split('.')
                y = y[0]
                y = re.findall(r'\d', y)
                string = ' '
                for j in y:
                    string += j
                y = float(string)
                # print(y)
                self.df.loc[index, 'Price'] = y
            else:
                y = i
                if '\n' in y:
                    y = y.split('\n')
                    y = y[0]
                y = y.split('.')
                y = y[0]
                y = re.sub(r'[\n₹,]', '', y)
                y = re.findall(r'\d', y)
                string = ' '
                for j in y:
                    string += j
                if len(string) == 0 or len(string) == 1:
                    string = 0
                else:
                    y = float(string)
                # print(y)
                self.df.loc[index, 'Price'] = y

            index += 1
            
        # cleaning review
        cleaned = []
        for i in self.df['Review']:
            i = str(i)
            x = re.sub(r"[^a-zA-Z\s.,!?']", "", i)
            y = re.sub(r'READ MORE', '', x)
            # y = re.sub(r'[^\x00-\x7F]+', '', x)
            cleaned.append(y)

        # tokenizing
        token = []
        stop_words = set(stopwords.words('english'))
        for i in cleaned:
            word_tokens = word_tokenize(i)
            # print(word_tokens)
            filtered_words = [word for word in word_tokens if not word.lower() in stop_words]
            word = ' '.join(filtered_words)
            words = re.sub('[^a-zA-Z0-9 \n]', '', word)
            print(words)
            token.append(words)
        self.df['Cleaned1'] = token
        
        ex = []
        for i in self.df.Link:
            match = re.search(r'www\.(.*?)\.(?:com|in)', i)
            extract = match.group(1)
            ex.append(extract)
        self.df['Website'] = ex

        return self.df

    # print('-------------')
    def predict(self, data):
        def get_sentiment(text):
            return TextBlob(text).sentiment.polarity
        self.df['sentiment'] = self.df['Cleaned1'].apply(get_sentiment)
        sentiment = self.df['sentiment']
        avg = self.df.groupby('Product')['sentiment'].mean().sort_values(ascending=False)[:1]
        best_product = avg.index
        polarity = avg.values
        price = self.df.loc[self.df['Product'] == best_product[0], 'Price'].values[0]
        link = self.df.loc[self.df['Product'] == best_product[0], 'Link'].values[0]
        rate = self.df.loc[self.df['Product'] == best_product[0], 'Avg Rating'].values[0]
        pro = df.Product.unique()
        pro_d = {}
        for i in pro:
        #     p = df.loc[df.Product == i]
        #     print(p.Link[])
            p = df.loc[df.Product == i]
            l = p.Link.unique()[0]
            match = re.search(r'www\.(.*?)\.(?:com|in)', l)
            if match:
                extract = match.group(1)
                pro_d[extract] = pro_d.get(extract, 0) + 1 
                
        sent = {}
        x = df.groupby('Website')['sentiment'].agg('mean')
        for i, j in x.items():
            sent[i] = j
                
        
        
        return df, pro_d, sent
    
    def group(self, df):
        groupby_of_all = df.groupby('Product')['Cleaned1'].apply(list)
        return groupby_of_all
    
   
    def aspect_based_sentiment(self, groupby_of_all):
        se = {}
        prods = groupby_of_all.index
        for i, j in enumerate(groupby_of_all.values):
            list = []
            for k in j:
                x = nltk.word_tokenize(k)
                y = nltk.pos_tag(x)
                list.append(y)
            se[prods[i]] = list

        products_review_with_sentiments = {}
        for i, j in se.items():
            d1 = {}
            for k in j:
                for l, m in enumerate(k):
                    if len(m[0]) > 4 and '...' not in m[0] and '....':
                        if m[1] == 'NN' or m[1] == 'VB':
                            aspect = m[0]
                            sentence = ''
                            if aspect not in d1.keys():
                                d1[aspect] = []
                            for n in k[l:]:
                                sentence += n[0] + ' '
                                if m[1] == 'JJ':
                                    break
                            x = d1[aspect]
                            x.append(sentence.strip())
                products_review_with_sentiments[i] = d1


        def sentiment(d):
            sid = SentimentIntensityAnalyzer()
            for i, j in d.items():
                #print(j)
                for k, l in j.items():
                    d2 = {}
                    for m in l:
                        sentiment = sid.polarity_scores(m)
                        d2[m] = sentiment

                    d[i][k] = d2
            return d

        x = sentiment(products_review_with_sentiments)


        # overall sentiment
        compound_aspect = {}
        for i in x.keys():
            asp = x[i]
            pos = 0
            neg = 0
            neu = 0 
            compound = 0
            count = 0
            for j in asp.keys():
                comment = x[i][j]
                for k in comment.values():
                    pos += k['pos']
                    neg += k['neg']
                    neu += k['neu']
                    compound += k['compound']
                    count+= 1
            try:
                compound_aspect[i] = compound/ count 
            except ZeroDivisionError:
                compound_aspect[i] = 0

        # each aspect overall sentiment
        pro_asp_info = {}
        for i in x.keys():
            asp = x[i]
            aspect_info = {}
            for j in asp.keys():
                pos = 0
                neg = 0
                neu = 0 
                compound = 0
                count = 0
                comment = x[i][j]
                #print(comment)
                #print(j)
                for k in comment.values():
                    pos += k['pos']
                    neg += k['neg']
                    neu += k['neu']
                    compound += k['compound']
                    count+= 1

                info = {'pos' : pos/ count, 'neg' : neg/ count, 'neu' : neu/ count, 'compound' : compound/ count}

                aspect_info[j] = info
            pro_asp_info[i] = aspect_info




        best_products = {}
        data = df.Price
        low = 1
        high = 10
        for i in range(10):

            if low == 1:
                lower_quartile = data.min()
            lower_quartile = np.percentile(data, low)
            high_quartile = np.percentile(data, high)

            # quartile
            closest_min_value = min(data, key=lambda x: abs(x - lower_quartile))
            closest_max_value = min(data, key=lambda x: abs(x - high_quartile))

            s1 = df.loc[(df.Price >= closest_min_value) & (df.Price <= closest_max_value)]

            pro = s1.Product.unique()
            fore ={}
            for i in pro:
                x1 = compound_aspect[i]
                fore[i] = x1
                #print(i)
            maxe = max(fore.values())
            #print("Products between {} - {} Quartile".format(low, high))

            pro = [k for k, v in fore.items() if v == maxe]
            #print(pro)

            prods = {}
            prods[pro[0]] = maxe
            best_products['Products between {} - {} Quartile'.format(low, high)] = prods 



            #print('==============')
            #print()
            low += 10
            high +=10
            if high > 100:
                break
        price = {}
        link = {}
        rating = {}
        #print(best_products)
        for k, v in best_products.items():
            price1 = {}
            link1 = {}
            rat = {}
            print(v)
            #print(list(v.keys()))
            a = [k for k, v1 in v.items()][0]
    #             for i in v.keys():
    #                 print(i)
            x1 = self.df.loc[df.Product == a]
            p = x1.Price.unique()[0]
            links = x1.Link.unique()[0]
            ratings = x1['Avg Rating'].unique()[0]
            price1[a] = p
            link1[a] = links
            rat[a] = ratings

            rating[k] = rat
            price[k] = price1
            link[k] = link1
            
        quartile1 = []
        quartile2 = []
        quartile3 = []
        quartile4 = []
        quartile5 = []
        quartile6 = []
        quartile7 = []
        quartile8 = []
        quartile9 = []
        quartile10 = []
        
        q1asp = []
        q2asp = []
        q3asp = []
        q4asp = []
        q5asp = []
        q6asp = []
        q7asp = []
        q8asp = []
        q9asp = []
        q10asp = []
        
        asp_for_best = {}
            for i, j in best_products.items():
                k = [l for l, v in j.items()][0]
                x = pro_asp_info[k]
                asp_for_best[k] = x

        for k, v in best_products.items():
            if '10' in k :
                key = [k1 for k1, v1 in v.items()][0]
                pol = [v1 for k1, v1 in v.items()][0]
                q1asp.append(asp_for_best[key])
                
                quartile1.append(key)
                quartile1.append(pol)

                pr = price[k]
                pri = [v1 for k1, v1 in pr.items()][0]
                quartile1.append(pri)

                li = link[k]
                links = [v1 for k1, v1 in li.items()][0]
                quartile1.append(links)

                ra = rating[k]
                ratings = [v1 for k1, v1 in ra.items()][0]
                quartile1.append(ratings)

            if '20' in k :
                key = [k1 for k1, v1 in v.items()][0]
                pol = [v1 for k1, v1 in v.items()][0]
                
                q2asp.append(asp_for_best[key])
                quartile2.append(key)
                quartile2.append(pol)

                pr = price[k]
                pri = [v1 for k1, v1 in pr.items()][0]
                quartile2.append(pri)

                li = link[k]
                links = [v1 for k1, v1 in li.items()][0]
                quartile2.append(links)

                ra = rating[k]
                ratings = [v1 for k1, v1 in ra.items()][0]
                quartile2.append(ratings)
            if '30' in k :
                key = [k1 for k1, v1 in v.items()][0]
                pol = [v1 for k1, v1 in v.items()][0]
                
                q3asp.append(asp_for_best[key])
                quartile3.append(key)
                quartile3.append(pol)

                pr = price[k]
                pri = [v1 for k1, v1 in pr.items()][0]
                quartile3.append(pri)

                li = link[k]
                links = [v1 for k1, v1 in li.items()][0]
                quartile3.append(links)

                ra = rating[k]
                ratings = [v1 for k1, v1 in ra.items()][0]
                quartile3.append(ratings)
            if '40' in k :
                key = [k1 for k1, v1 in v.items()][0]
                pol = [v1 for k1, v1 in v.items()][0]
                
                q4asp.append(asp_for_best[key])
                quartile4.append(key)
                quartile4.append(pol)

                pr = price[k]
                pri = [v1 for k1, v1 in pr.items()][0]
                quartile4.append(pri)

                li = link[k]
                links = [v1 for k1, v1 in li.items()][0]
                quartile4.append(links)

                ra = rating[k]
                ratings = [v1 for k1, v1 in ra.items()][0]
                quartile4.append(ratings)
            if '50' in k :
                key = [k1 for k1, v1 in v.items()][0]
                pol = [v1 for k1, v1 in v.items()][0]
                
                q5asp.append(asp_for_best[key])
                quartile5.append(key)
                quartile5.append(pol)

                pr = price[k]
                pri = [v1 for k1, v1 in pr.items()][0]
                quartile5.append(pri)

                li = link[k]
                links = [v1 for k1, v1 in li.items()][0]
                quartile5.append(links)

                ra = rating[k]
                ratings = [v1 for k1, v1 in ra.items()][0]
                quartile5.append(ratings)
            if '60' in k :
                key = [k1 for k1, v1 in v.items()][0]
                pol = [v1 for k1, v1 in v.items()][0]
                
                q6asp.append(asp_for_best[key])
                quartile6.append(key)
                quartile6.append(pol)

                pr = price[k]
                pri = [v1 for k1, v1 in pr.items()][0]
                quartile6.append(pri)

                li = link[k]
                links = [v1 for k1, v1 in li.items()][0]
                quartile6.append(links)

                ra = rating[k]
                ratings = [v1 for k1, v1 in ra.items()][0]
                quartile6.append(ratings)
            if '70' in k :
                key = [k1 for k1, v1 in v.items()][0]
                pol = [v1 for k1, v1 in v.items()][0]
                
                q7asp.append(asp_for_best[key])
                quartile7.append(key)
                quartile7.append(pol)

                pr = price[k]
                pri = [v1 for k1, v1 in pr.items()][0]
                quartile7.append(pri)

                li = link[k]
                links = [v1 for k1, v1 in li.items()][0]
                quartile7.append(links)

                ra = rating[k]
                ratings = [v1 for k1, v1 in ra.items()][0]
                quartile7.append(ratings)
            if '80' in k :
                key = [k1 for k1, v1 in v.items()][0]
                pol = [v1 for k1, v1 in v.items()][0]
                
                q8asp.append(asp_for_best[key])
                quartile8.append(key)
                quartile8.append(pol)

                pr = price[k]
                pri = [v1 for k1, v1 in pr.items()][0]
                quartile8.append(pri)

                li = link[k]
                links = [v1 for k1, v1 in li.items()][0]
                quartile8.append(links)

                ra = rating[k]
                ratings = [v1 for k1, v1 in ra.items()][0]
                quartile8.append(ratings)
            if '90' in k :
                key = [k1 for k1, v1 in v.items()][0]
                pol = [v1 for k1, v1 in v.items()][0]

                q9asp.append(asp_for_best[key])
                quartile9.append(key)
                quartile9.append(pol)

                pr = price[k]
                pri = [v1 for k1, v1 in pr.items()][0]
                quartile9.append(pri)

                li = link[k]
                links = [v1 for k1, v1 in li.items()][0]
                quartile9.append(links)

                ra = rating[k]
                ratings = [v1 for k1, v1 in ra.items()][0]
                quartile9.append(ratings)
            if '100' in k :
                key = [k1 for k1, v1 in v.items()][0]
                pol = [v1 for k1, v1 in v.items()][0]

                q10asp.append(asp_for_best[key])
                quartile10.append(key)
                quartile10.append(pol)

                pr = price[k]
                pri = [v1 for k1, v1 in pr.items()][0]
                quartile10.append(pri)

                li = link[k]
                links = [v1 for k1, v1 in li.items()][0]
                quartile10.append(links)

                ra = rating[k]
                ratings = [v1 for k1, v1 in ra.items()][0]
                quartile10.append(ratings)


            
            asp_for_best = {}
            for i, j in best_products.items():
                k = [l for l, v in j.items()][0]
                x = pro_asp_info[k]
                asp_for_best[k] = x

            

        return quartile1, quartile2, quartile3, quartile4, quartile5, quartile6, quartile7, quartile8, quartile9, quartile10, best_products, pro_asp_info 


        #groupby_of_all = df.groupby('Product')['Cleaned1'].apply(list)
    #bp, p, l, r = aspect_based_sentiment(groupby_of_all)
    

#     groupby_of_all = df.groupby('Product')['Cleaned1'].apply(list)
#     m1 = aspect_based_sentiment(groupby_of_all)
    
    
        #return best_product, polarity, link, price, rate


In [6]:
x = Cleancsv(df)
df = x.clean(df)
df, s1, s2 = x.predict(df)
y = x.group(df)
q1, q2, q3, q4, q5, q6, q7, q8, q9, q10, bp, asp= x.aspect_based_sentiment(y)

good product 
far good
Wonderful
Good
good performance
Great Product
Phone heats charging
Great product
nice
Nice experience
great 
Ok good
Great
Got best deal
Could better
Nice price good service
Good experience
good product
Great product service 
Nice shopping experience  Got best price 
Good
good
good
Good dealing
Great experience
Excellent
great thanks customer care guy Mr Seikh helped us thoroughly choose right product  Thumbs team Croma
Good
Good experience purchase
Satisfied
Nice product best service
Great deal
Product amazing
Excellent product
Good
product good
Nice product
Buying experience good
Good product good service Sagar Agrawal
Best service  Happy
Love 
good
Great
Good experience
good use
good product
Great
Thank Mr Amit helping giving good deal
good product Croma nice place purchase electronics supportive staff 
Nice
good
Amazing experience
Excellent
s bad
Nice work
Good experience great deal
good
Superb
good job
Good
device Alluminium body low refresh rate  compromise

product customer service excellent  Thanks
Nice shopping experience
Good love product
Great
Apple products always wow
ExcellentHappy use itVery comfortable cell phone
s fantastic experience
nice 
really good
Products good staff good  good experience Croma
Nice one
good experience definitely recommend family friends visit Croma 
iPhone purchased Croma nice use  got full support Croma executive existing backup etc 
great buying experience Croma  Infinity Mall  Malad branch  Sales person Zainab explained different offers  Definitely star Product purchased  offers Croma great buying experience 
Good Experience
Excellent superb camera
good
Good  Excellent
Product good  services sales person Ms Shivani Korke Cashiers billing department outstandingly efficient 
Excellent
excellent
Good nice
Good super
One best decision made nothing better phone pro max  
Product good
good deal thank Croma providing best service
happy phone
Great product
Awesome
Nice product
Waste money  Nothing great phone
Go

iPhone solid choice looking highquality smartphone experience  camera topnotch  performance smooth  design sleek modern  However  drawbacks consider comparing Android phonesOne biggest cons iPhone lack customization options  Apple strict policy apps features used devices  frustrating users want control pho 
Switched Xiaomi Pro heater means heats lot much cool device also used iPhone little bit better iPhone best device
s perfect mobile  Comfortable hands  Size good  Looks classy  s best
Apple iPhone great choice anyone looking reliable  fast stylish smartphone  inch OLED display delivers vibrant colors sharp details  powerful Bionic chip handles everything gaming multitasking ease  iPhone also improved camera system captures stunning photos videos lighting condition  thanks MP wideangle lens optical image stabilization  Smart HDR Action mode  iPh 
upgraded iPhone Plus  experienced huge improvement several areas daily usageI recommend buy iphone instead iphone various reasonsAdditional 

Great phone great capabilities Excellent camera Processor pro New blue colour awesome pro  pro   mini  Also new one iphone  One thing common  lag till date  one thing always said
love phone st time performance awesome wow camera quality battery everything awesome  prefer anyone use phone  Iphone using st time never getting problem  thank Flipkart providing low cost compared retail shops  Thank Flipkart Apple  love 
iPhone starlight beautiful design excellent camera great battery  think better previous year iPhone  Go
iPhone solid choice looking highquality smartphone experience  camera topnotch  performance smooth  design sleek modern  However  drawbacks consider comparing Android phonesOne biggest cons iPhone lack customization options  Apple strict policy apps features used devices  frustrating users want control pho 
Switched Xiaomi Pro heater means heats lot much cool device also used iPhone little bit better iPhone best device
s perfect mobile  Comfortable hands  Size good  Looks 

Simply awesome  Iphone already good enough  offers even improvement every way  biggest noticeable thing display  s crisp smooth colours vividAnd think everyone knows quality photos iPhones   Iphone offerd best ever camera photo quality yet iphonesvery happy purchase 
One best Smartphone buy right  Let s start Pros phone Alwayson screen gorgeous Exquisitely premium design Beautiful Hz display Incredible MP camera system Blistering performance Outstanding battery life Robust iOS software let s talk little Cons One heaviest phones buy Expensive
Great Phone price range  hz refresh rate feels smooth  great camera  big screen  good battery backup superb gaming  else anyone ask  Highly Recommended 
go  People comparing pro according s better one screen size much bigger battery life best compared series iphones even pro max ones Yes comes Hz refresh rate wo nt matter regular gamer  Coming camera course getting better quality pro potraits telephoto lens phones give quality sho 
absolute stunner

...

loved
Adidas iPhone pro max  Camera cinematic superb irrukey  Performance Verey level irukkey  iOS usuage amazing zing zingpoonakaluloading anthey
Perfect aspects
Nice superb camera qualityFeel premiumThanks flipkart delivery within daysGolden Colour awesome
Awesome
Awesome product
Thank
Great phone
Got delivered today  awesome phone
Everything premium never used phone like  camera quality unbelievable feel like using DSLRThanks Flipkart delivery days
first pro model  upgraded  Thank flipkart discount price compared shops 
Fantastic picture quality
one word awsm
Camera Awesome overall mobile look Cool
Awesome Best phone forever Good Gaming Experience Battery life Specially Camera superb
Mind blowing performance 
Best
Upgraded iPhone pro iPhone pro  awesome phone camera  Display quality good fast delivery 
Working well timely delivered
Overall nice 
One great device
Worth MoneyBrilliant life batteryThe wallpaper customisation updates awesomeThe camera clicks clear picEkart made super fa

Best budget Iphone till date go guys without second thought  Let explain guys Camera  Display  battery  performance  Camera price range comparison camera  youll love picture quality well video quality  Vlogger wanted iPhone k video front camera got phone happy Battery use phone roughly active social media person youtube channel  battery ba 
Really worth money  love  best phone ever 
Excellent camera Display touching Nice Smooth improvement phone new processor fast
Excellent camera  good performance  lag  lcd display also good  difference come watch movie prime Netflix  colours vibrant  see amoled display  charger box watts  takes hours charge  guys prepare buy watts charger  Finally android user  first Iapple phone  promise u guys iOS smooth  overall phone damn good  close eyes buy  thanks 
Value moneyIts awesome mobile phone world  Display good bright  Trust freinds r never regret Buying  go  love phone switch iphone x
Product fantastic great nay awesome cameraBattery management good 

Amazing camera quality expected  battery also good performance  Display  body  touch experience best Ive ever  Sound quality speakers sufficient  Apple known customer friendly security services see  processor best work  would definitely recommend everyone looking  go without delay  amazing thing delivery  Flipkart delivered span 
fabulous mobile look awesome performance unbeaten camera like dslr battery good think beast mobile
must buy looking upgrade previous generation iPhones  using XR still hold sometime upgrade model else phone must buy  Camera quality amazing wide angle something count upon  Performance wise amazing feels premium holding hand  big YES device  Go GB variant K videos occupy lots space storage get quickly  Try buy 
Great iPhone snappy experience apple kind  Upgraded iPhone  ProsCamera top class Battery top performed Chipset need say apple kind Security expect apple Display super bright industry leading colour accuracy super responsive Build quality expect apple stur

Excellent
iPhone awesomeCamera action mode Good batteryPrivacy
mobile good received quality product  defect software nt buy phone
Packaging good  see damaged corner  Amazon needs careful delivering expensive products  Hope happen 
best rounder  Thank Amazon delivering smoothly  Cheers 
Super
Awesome
wholesome experience ios  regrets shifting android ios
Genuine product

Heating issues thought ok day two still persists
Likes  mobile simply superb years using android device switched IPHONE precise performance butter smooth  cinematic camera awesome  look simple cool  Face unlock best among mobiles  Multitasking superbDislike  first day phone got hanged home screen alert hard reboot fortunately solution overcome  little bit worried saw  Front camera great per Iphone brand standard  Control center customizable s limited apps  Clear feature opened features androidOverall s good phone  got good deal along exchange offer 
Using last weekWorking smoothly cam quality also superb
Nice
Connect G 

{'APPLE iPhone 11 (Purple, 64 GB)': 0.6383666666666665}
{'Apple iPhone 14 (128 GB) - (Product) RED': 0.763929629629629}
{'APPLE iPhone 14 (Yellow, 128 GB)': 0.544376793248945}
{'APPLE iPhone 14 Plus (Blue, 128 GB)': 0.6469880733944953}
{'APPLE iPhone 14 Plus (Blue, 128 GB)': 0.6469880733944953}
{'Apple iPhone 14 (512 GB) - Blue': 0.7768}
{'APPLE iPhone 14 Plus (Purple, 512 GB)': 0.6469880733944953}
{'APPLE iPhone 14 Pro Max (Space Black, 128 GB)': 0.5161229885057468}
{'APPLE iPhone 14 Pro Max (Gold, 256 GB)': 0.5161229885057468}
{'Apple iPhone 14 Pro Max (1 TB) - Space Black': 0.7117636363636364}


In [15]:
asp_for_best = {}
for i, j in bp.items():
    k = [l for l, v in j.items()][0]
    x = asp[k]
    asp_for_best[k] = x
asp_for_best

{'APPLE iPhone 11 (Purple, 64 GB)': {'camera': {'pos': 0.37806249999999997,
   'neg': 0.019125000000000003,
   'neu': 0.6028125,
   'compound': 0.7401187499999999},
  'quality': {'pos': 0.3139090909090909,
   'neg': 0.034727272727272725,
   'neu': 0.6512727272727272,
   'compound': 0.754990909090909},
  'battery': {'pos': 0.3218888888888889,
   'neg': 0.006,
   'neu': 0.6721111111111111,
   'compound': 0.6663111111111111},
  'performance': {'pos': 0.29812500000000003,
   'neg': 0.01825,
   'neu': 0.683625,
   'compound': 0.699025},
  'experience': {'pos': 0.219,
   'neg': 0.0324,
   'neu': 0.7488,
   'compound': 0.46444},
  'customer': {'pos': 0.484, 'neg': 0.088, 'neu': 0.428, 'compound': 0.9168},
  'security': {'pos': 0.446, 'neg': 0.1, 'neu': 0.454, 'compound': 0.8654},
  'recommend': {'pos': 0.27, 'neg': 0.186, 'neu': 0.544, 'compound': 0.1002},
  'everyone': {'pos': 0.14, 'neg': 0.219, 'neu': 0.641, 'compound': -0.2755},
  'delay': {'pos': 0.342, 'neg': 0.207, 'neu': 0.45, 'compou

In [ ]:
q1asp = []

In [ ]:
pro = df.Product.unique()
pro_d = {}
for i in pro:
#     p = df.loc[df.Product == i]
#     print(p.Link[])
    p = df.loc[df.Product == i]
    l = p.Link.unique()[0]
    match = re.search(r'www\.(.*?)\.(?:com|in)', l)
    if match:
        extract = match.group(1)
        pro_d[extract] = pro_d.get(extract, 0) + 1 

In [ ]:
pro_d

In [ ]:
ex = []
for i in df.Link:
    match = re.search(r'www\.(.*?)\.(?:com|in)', i)
    extract = match.group(1)
    ex.append(extract)
df['Website'] = ex


In [ ]:
sent = {}
x = df.groupby('Website')['sentiment'].agg('mean')
for i, j in x.items():
    sent[i] = j

In [ ]:
quartile_1 = []
quartile_2 = []
quartile_3 = []
quartile_4 = []
quartile_5 = []
quartile_6 = []
quartile_7 = []
quartile_8 = []
quartile_9 = []
quartile_10 = []


for k, v in bp.items():
    if '10' in k :
        key = [k1 for k1, v1 in v.items()][0]
        pol = [v1 for k1, v1 in v.items()][0]

        quartile1.append(key)
        quartile1.append(pol)
        
        pr = p[k]
        price = [v1 for k1, v1 in pr.items()][0]
        quartile1.append(price)
        
        li = l[k]
        links = [v1 for k1, v1 in li.items()][0]
        quartile1.append(links)
        
        ra = r[k]
        rating = [v1 for k1, v1 in ra.items()][0]
        quartile1.append(rating)
        
    if '20' in k :
        key = [k1 for k1, v1 in v.items()][0]
        pol = [v1 for k1, v1 in v.items()][0]

        quartile2.append(key)
        quartile2.append(pol)
        
        pr = p[k]
        price = [v1 for k1, v1 in pr.items()][0]
        quartile2.append(price)
        
        li = l[k]
        links = [v1 for k1, v1 in li.items()][0]
        quartile2.append(links)
        
        ra = r[k]
        rating = [v1 for k1, v1 in ra.items()][0]
        quartile2.append(rating)
    if '30' in k :
        key = [k1 for k1, v1 in v.items()][0]
        pol = [v1 for k1, v1 in v.items()][0]

        quartile3.append(key)
        quartile3.append(pol)
        
        pr = p[k]
        price = [v1 for k1, v1 in pr.items()][0]
        quartile3.append(price)
        
        li = l[k]
        links = [v1 for k1, v1 in li.items()][0]
        quartile3.append(links)
        
        ra = r[k]
        rating = [v1 for k1, v1 in ra.items()][0]
        quartile3.append(rating)
    if '40' in k :
        key = [k1 for k1, v1 in v.items()][0]
        pol = [v1 for k1, v1 in v.items()][0]

        quartile4.append(key)
        quartile4.append(pol)
        
        pr = p[k]
        price = [v1 for k1, v1 in pr.items()][0]
        quartile4.append(price)
        
        li = l[k]
        links = [v1 for k1, v1 in li.items()][0]
        quartile4.append(links)
        
        ra = r[k]
        rating = [v1 for k1, v1 in ra.items()][0]
        quartile4.append(rating)
    if '50' in k :
        key = [k1 for k1, v1 in v.items()][0]
        pol = [v1 for k1, v1 in v.items()][0]

        quartile5.append(key)
        quartile5.append(pol)
        
        pr = p[k]
        price = [v1 for k1, v1 in pr.items()][0]
        quartile5.append(price)
        
        li = l[k]
        links = [v1 for k1, v1 in li.items()][0]
        quartile5.append(links)
        
        ra = r[k]
        rating = [v1 for k1, v1 in ra.items()][0]
        quartile5.append(rating)
    if '60' in k :
        key = [k1 for k1, v1 in v.items()][0]
        pol = [v1 for k1, v1 in v.items()][0]

        quartile6.append(key)
        quartile6.append(pol)
        
        pr = p[k]
        price = [v1 for k1, v1 in pr.items()][0]
        quartile6.append(price)
        
        li = l[k]
        links = [v1 for k1, v1 in li.items()][0]
        quartile6.append(links)
        
        ra = r[k]
        rating = [v1 for k1, v1 in ra.items()][0]
        quartile6.append(rating)
    if '70' in k :
        key = [k1 for k1, v1 in v.items()][0]
        pol = [v1 for k1, v1 in v.items()][0]

        quartile7.append(key)
        quartile7.append(pol)
        
        pr = p[k]
        price = [v1 for k1, v1 in pr.items()][0]
        quartile7.append(price)
        
        li = l[k]
        links = [v1 for k1, v1 in li.items()][0]
        quartile7.append(links)
        
        ra = r[k]
        rating = [v1 for k1, v1 in ra.items()][0]
        quartile7.append(rating)
    if '80' in k :
        key = [k1 for k1, v1 in v.items()][0]
        pol = [v1 for k1, v1 in v.items()][0]

        quartile8.append(key)
        quartile8.append(pol)
        
        pr = p[k]
        price = [v1 for k1, v1 in pr.items()][0]
        quartile8.append(price)
        
        li = l[k]
        links = [v1 for k1, v1 in li.items()][0]
        quartile8.append(links)
        
        ra = r[k]
        rating = [v1 for k1, v1 in ra.items()][0]
        quartile8.append(rating)
    if '90' in k :
        key = [k1 for k1, v1 in v.items()][0]
        pol = [v1 for k1, v1 in v.items()][0]

        quartile9.append(key)
        quartile9.append(pol)
        
        pr = p[k]
        price = [v1 for k1, v1 in pr.items()][0]
        quartile9.append(price)
        
        li = l[k]
        links = [v1 for k1, v1 in li.items()][0]
        quartile9.append(links)
        
        ra = r[k]
        rating = [v1 for k1, v1 in ra.items()][0]
        quartile9.append(rating)
    if '100' in k :
        key = [k1 for k1, v1 in v.items()][0]
        pol = [v1 for k1, v1 in v.items()][0]

        quartile10.append(key)
        quartile10.append(pol)
        
        pr = p[k]
        price = [v1 for k1, v1 in pr.items()][0]
        quartile10.append(price)
        
        li = l[k]
        links = [v1 for k1, v1 in li.items()][0]
        quartile10.append(links)
        
        ra = r[k]
        rating = [v1 for k1, v1 in ra.items()][0]
        quartile10.append(rating)

In [ ]:
quartile9

In [ ]:
#p, pol, l, pr, ra = x.predict(df)


In [ ]:
# product as key and its tokenize pos as key


groupby_of_all = df.groupby('Product')['Cleaned1'].apply(list)
se = {}
prods = groupby_of_all.index
for i, j in enumerate(groupby_of_all.values):
    list = []
    for k in j:
        x = nltk.word_tokenize(k)
        y = nltk.pos_tag(x)
        list.append(y)
    se[prods[i]] = list

In [ ]:
# # optional - taking only one product
    
# d = se['APPLE iPhone 11 (Black, 128 GB)']
# x = {}
# x['APPLE iPhone 11 (Black, 128 GB)'] = d

In [ ]:

# like the dictionary says, but without sentiments
products_review_with_sentiments = {}
for i, j in se.items():
    d1 = {}
    for k in j:
        for l, m in enumerate(k):
            if len(m[0]) > 4 and '...' not in m[0] and '....':
                if m[1] == 'NN' or m[1] == 'VB':
                    aspect = m[0]
                    sentence = ''
                    if aspect not in d1.keys():
                        d1[aspect] = []
                    for n in k[l:]:
                        sentence += n[0] + ' '
                        if m[1] == 'JJ':
                            break
                    x = d1[aspect]
                    x.append(sentence.strip())
        products_review_with_sentiments[i] = d1

In [ ]:
# calculating sentiment of the aspect sentence
def sentiment(d):
    sid = SentimentIntensityAnalyzer()
    for i, j in d.items():
        #print(j)
        for k, l in j.items():
            d2 = {}
            for m in l:
                sentiment = sid.polarity_scores(m)
                d2[m] = sentiment
          
            d[i][k] = d2
    return d
            
x = sentiment(products_review_with_sentiments)


In [ ]:
# overall sentiment

compound_aspect = {}
for i in x.keys():
    asp = x[i]
    pos = 0
    neg = 0
    neu = 0 
    compound = 0
    count = 0
    for j in asp.keys():
        comment = x[i][j]
        for k in comment.values():
            pos += k['pos']
            neg += k['neg']
            neu += k['neu']
            compound += k['compound']
            count+= 1
    try:
        compound_aspect[i] = compound/ count 
    except ZeroDivisionError:
        compound_aspect[i] = 0

In [ ]:
compound_aspect

In [ ]:
# each aspect overall sentiment

pro_asp_info = {}
for i in x.keys():
    asp = x[i]
    aspect_info = {}
    for j in asp.keys():
        pos = 0
        neg = 0
        neu = 0 
        compound = 0
        count = 0
        comment = x[i][j]
        #print(comment)
        #print(j)
        for k in comment.values():
            pos += k['pos']
            neg += k['neg']
            neu += k['neu']
            compound += k['compound']
            count+= 1
        
        info = {'pos' : pos/ count, 'neg' : neg/ count, 'neu' : neu/ count, 'compound' : compound/ count}
            
        aspect_info[j] = info
    pro_asp_info[i] = aspect_info
        
pro_asp_info

In [ ]:
min_price = df.loc[df.Price == df.Price.min()].index
max_price = df.loc[df.Price == df.Price.max()].index
mean_price = df.loc[df.Price == df.Price.mean()].index

mean = df.Price.mean()
std = df['Price'].std()

max_rating = df.loc[df["Avg Rating"] == df['Avg Rating'].max()].index


num_std = 0.09

ctm = df[(df['Price'] >= (mean - (num_std * std))) & (df['Price'] <= (mean + (num_std * std)))]

In [ ]:
ctm.Product.unique()

In [ ]:
pro_asp_info[key[0]]

In [ ]:
max_value = max(compound_aspect.values())
print(max_value)

In [ ]:
key = [k for k, v in compound_aspect.items() if v == max_value]
key


In [ ]:
df

In [ ]:

def aspect_based_sentiment(groupby_of_all):
    
    se = {}
    prods = groupby_of_all.index
    for i, j in enumerate(groupby_of_all.values):
        list = []
        for k in j:
            x = nltk.word_tokenize(k)
            y = nltk.pos_tag(x)
            list.append(y)
        se[prods[i]] = list
        
    products_review_with_sentiments = {}
    for i, j in se.items():
        d1 = {}
        for k in j:
            for l, m in enumerate(k):
                if len(m[0]) > 4 and '...' not in m[0] and '....':
                    if m[1] == 'NN' or m[1] == 'VB':
                        aspect = m[0]
                        sentence = ''
                        if aspect not in d1.keys():
                            d1[aspect] = []
                        for n in k[l:]:
                            sentence += n[0] + ' '
                            if m[1] == 'JJ':
                                break
                        x = d1[aspect]
                        x.append(sentence.strip())
            products_review_with_sentiments[i] = d1
            
            
    def sentiment(d):
        sid = SentimentIntensityAnalyzer()
        for i, j in d.items():
            #print(j)
            for k, l in j.items():
                d2 = {}
                for m in l:
                    sentiment = sid.polarity_scores(m)
                    d2[m] = sentiment

                d[i][k] = d2
        return d
            
    x = sentiment(products_review_with_sentiments)
    
    
    # overall sentiment
    compound_aspect = {}
    for i in x.keys():
        asp = x[i]
        pos = 0
        neg = 0
        neu = 0 
        compound = 0
        count = 0
        for j in asp.keys():
            comment = x[i][j]
            for k in comment.values():
                pos += k['pos']
                neg += k['neg']
                neu += k['neu']
                compound += k['compound']
                count+= 1
        try:
            compound_aspect[i] = compound/ count 
        except ZeroDivisionError:
            compound_aspect[i] = 0
            
    # each aspect overall sentiment
    pro_asp_info = {}
    for i in x.keys():
        asp = x[i]
        aspect_info = {}
        for j in asp.keys():
            pos = 0
            neg = 0
            neu = 0 
            compound = 0
            count = 0
            comment = x[i][j]
            #print(comment)
            #print(j)
            for k in comment.values():
                pos += k['pos']
                neg += k['neg']
                neu += k['neu']
                compound += k['compound']
                count+= 1

            info = {'pos' : pos/ count, 'neg' : neg/ count, 'neu' : neu/ count, 'compound' : compound/ count}

            aspect_info[j] = info
        pro_asp_info[i] = aspect_info
        
        
    
    
    best_products = {}
    data = df.Price
    low = 1
    high = 10
    for i in range(10):

        if low == 1:
            lower_quartile = data.min()
        lower_quartile = np.percentile(data, low)
        high_quartile = np.percentile(data, high)

        # quartile
        closest_min_value = min(data, key=lambda x: abs(x - lower_quartile))
        closest_max_value = min(data, key=lambda x: abs(x - high_quartile))

        s1 = df.loc[(df.Price >= closest_min_value) & (df.Price <= closest_max_value)]

        pro = s1.Product.unique()
        fore ={}
        for i in pro:
            x1 = compound_aspect[i]
            fore[i] = x1
            #print(i)
        maxe = max(fore.values())
        #print("Products between {} - {} Quartile".format(low, high))

        pro = [k for k, v in fore.items() if v == maxe]
        #print(pro)

        prods = {}
        prods[pro[0]] = maxe
        best_products['Products between {} - {} Quartile'.format(low, high)] = prods 



        #print('==============')
        #print()
        low += 10
        high +=10
        if high > 100:
            break
    price = {}
    link = {}
    rating = {}
    #print(best_products)
    for k, v in best_products.items():
        price1 = {}
        link1 = {}
        rat = {}
        print(v)
        #print(list(v.keys()))
        a = [k for k, v1 in v.items()][0]
#             for i in v.keys():
#                 print(i)
        x1 = df.loc[df.Product == a]
        p = x1.Price.unique()[0]
        links = x1.Link.unique()[0]
        ratings = x1['Avg Rating'].unique()[0]
        price1[a] = p
        link1[a] = links
        rat[a] = ratings

        rating[k] = rat
        price[k] = price1
        link[k] = link1
        
        
    
    
    return best_products, price, link, rating


groupby_of_all = df.groupby('Product')['Cleaned1'].apply(list)
bp, p, l, r = aspect_based_sentiment(groupby_of_all)
    

In [ ]:
r

In [ ]:
d[0]

In [ ]:
price = {}
link = {}
rating = {}
for k, v in m1.items():
    price1 = {}
    link1 = {}
    rat = {}
    print(v)
    a = list(v.keys())[0]
    x1 = df.loc[df.Product == a]
    p = x1.Price.unique()[0]
    links = x1.Link.unique()[0]
    ratings = x1['Avg Rating'].unique()[0]
    price1[a] = p
    link1[a] = links
    rat[a] = ratings
    
    rating[k] = rat
    price[k] = price1
    link[k] = link1

In [ ]:
a = list(m1[j].keys())[0]

In [ ]:
a

In [ ]:
x1 = df.loc[df.Product == a]
price = x1.Price
link = x1.Link

In [ ]:
link.unique()[0]

In [ ]:
m1

In [ ]:
data = df.Price
lower_quartile = np.percentile(data, 10)
high_quartile = np.percentile(data, 20)
closest_min_value = min(data, key=lambda x: abs(x - lower_quartile))
closest_max_value = min(data, key=lambda x: abs(x - high_quartile))

s1 = df.loc[(df.Price >= closest_min_value) & (df.Price <= closest_max_value)]

In [ ]:
# main 

best_products = {}
data = df.Price
low = 1
high = 10
for i in range(10):
    
    if low == 1:
        lower_quartile = data.min()
    lower_quartile = np.percentile(data, low)
    high_quartile = np.percentile(data, high)
    
    # quartile
    closest_min_value = min(data, key=lambda x: abs(x - lower_quartile))
    closest_max_value = min(data, key=lambda x: abs(x - high_quartile))

    s1 = df.loc[(df.Price >= closest_min_value) & (df.Price <= closest_max_value)]
    
    pro = s1.Product.unique()
    fore ={}
    for i in pro:
        x1 = m1[i]
        fore[i] = x1
        #print(i)
    maxe = max(fore.values())
    print("Products between {} - {} Quartile".format(low, high))

    pro = [k for k, v in fore.items() if v == maxe]
    print(pro)
    
    prods = {}
    prods[pro[0]] = maxe
    best_products['Products between {} - {} Quartile'.format(low, high)] = prods 
    
    
        
    print('==============')
    #print()
    low += 10
    high +=10
    if high > 100:
        break
        
    
    
    

In [ ]:
best_products

In [ ]:
x = group(df)

In [ ]:
x

In [ ]:
import pandas as pd

df = pd.read_csv('Iphone14Reviews.csv')
df

In [ ]:
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC

nlp = spacy.load("en_core_web_sm")

def extract_aspects(sentence):
    doc = nlp(sentence)
    aspects = []

    # Rule-based aspect extraction
    aspect_rules = {
        "food": ["food", "cuisine", "dishes", "menu"],
        "service": ["service", "staff", "waiter", "waitress"],
        "ambience": ["ambience", "atmosphere", "decor"],
    }

    for token in doc:
        for aspect, keywords in aspect_rules.items():
            if token.text.lower() in keywords:
                aspects.append(aspect)

    # Machine learning-based aspect extraction
    ml_aspects = machine_learning_extraction([sentence])

    # Combine rule-based and ML-based aspects
    aspects += ml_aspects

    return aspects

def machine_learning_extraction(sentences):
    # Prepare training data
    train_data = [
        ("The food was amazing, but the service was slow.", ["food", "service"]),
        ("The atmosphere was cozy, but the food was disappointing.", ["ambience", "food"]),
        # Add more training examples...
    ]
    train_sentences, train_aspects = zip(*train_data)

    # Vectorize the training data
    vectorizer = CountVectorizer()
    X_train = vectorizer.fit_transform(train_sentences)

    # Train the classifier
    classifier = LinearSVC()
    classifier.fit(X_train, train_aspects)

    # Vectorize the input sentences
    X_test = vectorizer.transform(sentences)

    # Predict aspects using the trained classifier
    predicted_aspects = classifier.predict(X_test)

    return predicted_aspects.tolist()

# Example usage
sentence = "The food was delicious, but the service was slow."
aspects = extract_aspects(sentence)
print(aspects)


In [ ]:
import os
from oscarcorpus import load_corpus

# Set the path to the OSCAR Corpus data directory
oscar_data_dir = '/path/to/oscar/data'

# Load the Arabic subcorpus from the OSCAR Corpus
arabic_corpus = load_corpus(oscar_data_dir, languages=['ar'])

# Iterate through the sentences in the Arabic corpus
for sentence in arabic_corpus.sentences():
    # Access the text of the sentence
    text = sentence.text

    # Process the sentence as needed
    # (e.g., perform tokenization, linguistic analysis, etc.)

    # Print the processed text
    print(text)


In [ ]:
pip install oscarcorpus

In [ ]:
import os
from tqdm import tqdm

# Set the path to the OSCAR Corpus data directory
oscar_data_dir = '/path/to/oscar/data'

# Define the language code for Arabic
language_code = 'ar'

# Iterate through the files in the Arabic subdirectory
for root, dirs, files in os.walk(os.path.join(oscar_data_dir, language_code)):
    for filename in tqdm(files):
        file_path = os.path.join(root, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            
            # Process the text as needed
            # (e.g., perform tokenization, linguistic analysis, etc.)
            
            # Print the processed text
            print(text)


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

# Load the MTB-Arabic dataset
dataset = hub.load("https://tfhub.dev/google/mtb-arabic/1")

# Split the dataset into training and test sets
train_dataset, test_dataset = dataset.split(0.8)

# Create a model
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(dataset.vocab_size, 128),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(dataset.vocab_size, activation="softmax")
])

# Train the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(train_dataset, epochs=10)

# Evaluate the model
model.evaluate(test_dataset)


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Arabic text data
texts = [
    "مرحبا بك في العالم",
    "أهلا وسهلا",
    "كيف حالك؟",
    "ماذا تفعل؟"
]

# Labels corresponding to the texts
labels = [0, 1, 1, 0]

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(texts)

# Pad sequences to ensure uniform length
max_length = max(len(sequence) for sequence in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# Create a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((padded_sequences, labels))

# Shuffle and batch the dataset
batch_size = 2
dataset = dataset.shuffle(len(texts)).batch(batch_size)

# Example usage: Iterate over the dataset
for batch in dataset:
    padded_text, label = batch
    print("Padded Text:\n", padded_text)
    print("Label:\n", label)
    print()


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

# Load the MTB-Arabic dataset
dataset = hub.load("https://tfhub.dev/google/mtb-arabic/1")

# Split the dataset into training and test sets
train_dataset, test_dataset = dataset.split(0.8)

# Create a model
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(dataset.vocab_size, 128),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(dataset.vocab_size, activation="softmax")
])

# Train the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(train_dataset, epochs=10)

# Evaluate the model
model.evaluate(test_dataset)
